# Generating Gaussian sims

## Importing Modules

In [ ]:
import os                                                                     
from threadpoolctl import threadpool_limits
import healpy as hp
import numpy as np
from tqdm import tqdm
from astropy.io import fits as pyfits
import random
import pymp
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import importlib
import sys
import multiprocessing as mp
from math import sqrt
from math import radians
import gc
import subprocess
gc.disable()
#######################################################################################################
from cmbaberdopp_beta0p8 import * # cmbboost its a module created w/ the functions needed for this work          #
                       # remember to change the path of maps on dipole_doppler_load_low_res function  #
#######################################################################################################
def c_nthreads_limit(n): # Function to change number of threads used on imported C/C++ functions
    threadpool_limits(limits=n, user_api='blas')
    threadpool_limits(limits=n, user_api='openmp')

In [ ]:
nsims = 3072

nthreads= 4
c_nthreads_limit(nthreads)
nprocesses = 3

gaussian_alms_dir = ''

nside_rot = 2 # setting the directions do you want to apply the modulations

clTT = hp.read_cl("cl_TT_planck_2019.fits")
clTE = hp.read_cl("cl_TE_planck_2019.fits")
clEE = hp.read_cl("cl_EE_planck_2019.fits")
clBB = np.zeros(clTT.size)

If you want a high number of directions it will useful to parallelize the for loop using multiprocessing - but pay attention on RAM usage.

In [ ]:
def gaussian_loop(i):
    start = time.time()
    alm_T, alm_E, alm_B = hp.synalm((clTT, clTE, clEE, clBB),lmax=2048,new=False,verbose=False)
    hp.write_alm(gaussian_alms_dir+'gaussian_alm_T_'+str(i)+'.fits',alm_T,out_dtype=np.float64)
    hp.write_alm(gaussian_alms_dir+'gaussian_alm_E_'+str(i)+'.fits',alm_E,out_dtype=np.float64)
    end = time.time()
    print('sim: ',i,' time: ',end-start)
pool = mp.Pool(processes=nprocesses)
pool.map(gaussian_loop, range(nsims))
pool.terminate()
pool.join()